In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/chronic-kidney-disease/kidney_disease_train.csv')
df.head()

In [ ]:
df.info()

In [ ]:
missing_values = (df.isnull().sum()/len(df)) * 100                  # Visualization of missing values in the dataset
missing_values.sort_values(ascending=False,inplace=True)
plt.barh(y=missing_values.index,width= missing_values.values)

In [ ]:
df['classification'].replace({'ckd':1,'notckd':0},inplace=True) # replace classification with numbers

In [ ]:
df.select_dtypes('object').nunique()  # drop object columns  wc,rc,dm,cad many unique values

In [ ]:
df.drop(columns=['wc','rc','dm','cad','id'],inplace=True)

In [ ]:
df.head()

In [ ]:
dummies = list(df.select_dtypes('object').columns) # create dummies and drop drop_first !
dummies

In [ ]:
df = pd.get_dummies(df,columns=dummies)
df.head()

In [ ]:
correlation = abs(df.corr().sort_values(by='classification')['classification']) # check correlation of features with  "classification2
correlation

In [ ]:
select = list(correlation[correlation > 0.3].index) # select features with correlation higher then 0.3
select

In [ ]:
df[select].isnull().sum() # check missing values

In [ ]:
df.dropna(how='any',inplace=True)  #  drop all rows with nan, keep it easy without any imputation

In [ ]:
X = df.drop(columns='classification')    # Split label and datsets
y = df['classification']

In [ ]:
X

### Split in train and test, normalization

In [ ]:
from sklearn.model_selection import train_test_split    
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

## model selection

In [ ]:
import tensorflow as tf
from  tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.constraints import max_norm

In [ ]:
model = Sequential()

model.add(Dense(28,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(14,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(6,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(3,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1,activation='sigmoid'))
model.add(Dropout(0.2))

model.compile(loss='binary_crossentropy',optimizer='adam')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping  

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
model.fit(x=X_train, 
          y=y_train,
          epochs=2000,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop]
          )

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

In [ ]:
predictions = model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,predictions))  # 100 % ist not possible, what is wrong ?

In [ ]:
print(confusion_matrix(y_test,predictions))     #  confusion matrix